<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#LGBM-training" data-toc-modified-id="LGBM-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LGBM training</a></span><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare</a></span></li><li><span><a href="#Core" data-toc-modified-id="Core-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Core</a></span></li></ul></li></ul></div>

# Load

In [1]:
import taiko as tk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb

SONG = 1
WHO = 7
TRAIN_ORDER = 3

Using TensorFlow backend.


In [2]:
pfs = []
for drummer in tqdm(range(1, 8 + 1)):
    ppf = []
    for order_id in range(1, 3 + 1):
        ppf.append(tk.get_performance(drummer, SONG, order_id).event_primitive_df)
    pfs.append(ppf)

100%|██████████| 8/8 [08:00<00:00, 60.12s/it]


In [3]:
def soso(who_id, song_id):
    sfs = []
    for pf_order in range(1, 3 + 1):
        img_scores, _ = tk.get_scores_with_timestamps(who_id, song_id, pf_order)
        sfs.append(img_scores[-1])
    return sfs

In [4]:
sfs = []
for drumer in range(1, 8 + 1):
    sfs.append(soso(drumer, SONG))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
3072/3900 [======================>.......] - ETA: 0s

In [5]:
sfs[0][0]

243400

In [6]:
pfs[0][2]

,hit_type,R_AAI,R_AVI,R_ASMA,R_GAI,R_GVI,R_GSMA,R_AAE,R_ARE,R_MAMI,...,L_AXYCORR,L_AYZCORR,L_AZXCORR,L_GXYCORR,L_GYZCORR,L_GZXCORR,L1,L2,R1,R2
0,1,0.458405,0.133632,0.663308,40.484167,467.567143,61.566192,0.343767,2106.534920,0.302460,...,-0.387950,-0.707634,0.638282,-0.008275,0.450733,0.764722,0,0,0,0
1,0,0.099319,0.003155,0.142458,53.744552,109.898802,85.183958,0.013019,2998.375689,0.072609,...,0.342950,0.456435,0.484237,0.361747,0.689131,-0.334058,1,0,0,0
2,0,0.109905,0.000113,0.181583,18.301596,31.484895,26.038083,0.012192,366.433295,0.109099,...,-0.861537,0.741859,-0.938416,-0.663970,-0.919219,0.781575,0,1,0,1
3,0,0.129120,0.000397,0.156024,11.975427,2.007300,18.444667,0.017069,145.418158,0.126321,...,0.981657,-0.873096,-0.860675,0.929933,-0.682735,-0.540015,0,0,1,0
4,1,0.148046,0.000482,0.196875,13.130628,20.818286,20.115167,0.022399,193.231670,0.153234,...,0.622447,-0.183582,-0.651717,0.588803,0.140461,0.094272,0,0,0,0
5,0,0.109907,0.000628,0.164090,11.456303,9.326654,17.065923,0.012708,140.573532,0.104815,...,-0.222814,0.817765,-0.624309,-0.139521,-0.340879,0.691003,1,0,0,0
6,0,0.113657,0.000403,0.146083,18.795879,20.442007,28.714833,0.013321,373.727071,0.117048,...,0.574720,0.366917,-0.461036,-0.937800,-0.929987,0.963170,0,1,0,1
7,0,0.049250,0.000282,0.079667,29.328893,12.231044,49.361542,0.002708,872.415034,0.048813,...,0.918398,-0.373898,-0.246744,-0.552946,0.423405,-0.577769,0,0,1,0
8,1,0.220046,0.018341,0.317792,49.159211,505.297197,74.524500,0.066761,2921.925256,0.165814,...,-0.709432,0.704623,-0.542153,0.620837,0.383911,-0.365007,0,0,0,1
9,0,0.079653,0.000912,0.124875,24.596001,15.287398,38.988375,0.007257,620.250646,0.069283,...,-0.538945,0.889621,-0.274946,0.135624,-0.274117,-0.882012,1,0,1,0


# LGBM training

## Prepare

In [7]:
def prepare(train_who, train_porder, valid_who, valid_porder):
    train_df = pfs[train_who - 1][train_porder - 1]
    x = train_df.drop(['hit_type'], axis=1)
    y = train_df['hit_type']
    
    valid_df = pfs[valid_who - 1][valid_porder - 1]
    x_valid = valid_df.drop(['hit_type'], axis=1)
    y_valid = valid_df['hit_type']
    
    return x, y, x_valid, y_valid

In [8]:
params = dict({
    'learning_rate': 0.5,
    'application': 'multiclass',
    'num_classes': 3,
#     'min_data_in_leaf': 5,
#     'max_depth': 8,
    'num_leaves': 2 ** 3,
    'verbosity': 0,
    'metric': 'multi_error'
})

grid_params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [8, 10],
}

In [9]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    return cm

In [10]:
def get_multi_error(y, pred_y):
    cnf_matrix = confusion_matrix(y, pred_y)
    
    # Plot normalized confusion matrix
#     plt.figure(figsize=(10, 10))
    cm = plot_confusion_matrix(cnf_matrix, classes=[0, 1, 2], normalize=True,
                          title='Normalized confusion matrix')
    return sum([1 - cm[i][i] for i in range(cm.shape[0])]) / cm.shape[0]

## Core

In [11]:
result_df = pd.DataFrame(columns=['train_who', 'train_porder', 'train_score', 'valid_who', 'valid_porder', 'valid_score', 'test_who', 'test_porder', 'test_score', 'multi_error'])
id_ = 0

for train_who in range(1, 8 + 1):
    for train_porder in range(1, 3 + 1):
        for valid_who in range(1, 8 + 1):
            for valid_porder in range(1, 3 + 1):
                x, y, x_valid, y_valid = prepare(train_who, train_porder, valid_who, valid_porder)
            
                train_set = lgb.Dataset(x, y)
                valid_set = lgb.Dataset(x_valid, y_valid, free_raw_data=False)

                watchlist = [valid_set]

                model = lgb.train(params, train_set=train_set, valid_sets=watchlist, num_boost_round=200, verbose_eval=50, early_stopping_rounds=100)

                for test_who in range(1, 8 + 1):
                    for test_porder in range(1, 3 + 1):

                        x_test = pfs[test_who - 1][test_porder - 1].drop(['hit_type'], axis=1)
                        y_true = pfs[test_who - 1][test_porder - 1]['hit_type']

                        y_pred = model.predict(x_test, num_iteration=model.best_iteration)
                        predictions = []
                        for xx in y_pred:
                            predictions.append(np.argmax(xx))
                        y_test = pd.Series(data=predictions)

                        multierror = '%.4f' % get_multi_error(y_true, y_test)

    #                     train_str = '%d,%d' % (train_who, train_porder)
                        train_score = sfs[train_who - 1][train_porder - 1]

    #                     valid_str = '%d,%d' % (train_who, valid_porder)
                        valid_score = sfs[valid_who - 1][valid_porder - 1]

    #                     test_str = '%d,%d' % (test_who, test_porder)
                        test_score = sfs[test_who - 1][test_porder - 1]

                        result_df.loc[id_] = [train_who, train_porder, train_score, valid_who, valid_porder, valid_score, test_who, test_porder, test_score, multierror]

                        id_ += 1

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0
[100]	valid_0's multi_error: 0
Early stopping, best iteration is:
[11]	valid_0's multi_error: 0
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0526316
[100]	valid_0's multi_error: 0.0581717
Early stopping, best iteration is:
[35]	valid_0's multi_error: 0.0443213
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0720222
[100]	valid_0's multi_error: 0.066482
Early stopping, best iteration is:
[37]	valid_0's multi_error: 0.0637119
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.166205
[100]	valid_0's multi_error: 0.155125
[150]	valid_0's multi_error: 0.163435
Early stopping, best iteration is:
[64]	valid_0's multi_error: 0.146814
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.174515
[100]	valid_0's multi_error: 0.174515


Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.152355
[100]	valid_0's multi_error: 0.144044
Early stopping, best iteration is:
[43]	valid_0's multi_error: 0.130194
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.138504
[100]	valid_0's multi_error: 0.141274
[150]	valid_0's multi_error: 0.138504
Early stopping, best iteration is:
[54]	valid_0's multi_error: 0.130194
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.163435
[100]	valid_0's multi_error: 0.160665
Early stopping, best iteration is:
[4]	valid_0's multi_error: 0.144044
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.105263
[100]	valid_0's multi_error: 0.108033
Early stopping, best iteration is:
[38]	valid_0's multi_error: 0.099723
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0969529
[100]	valid_0's multi_

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.146814
[100]	valid_0's multi_error: 0.157895
Early stopping, best iteration is:
[29]	valid_0's multi_error: 0.135734
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0831025
[100]	valid_0's multi_error: 0.0886427
Early stopping, best iteration is:
[32]	valid_0's multi_error: 0.0637119
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0858726
[100]	valid_0's multi_error: 0.0941828
Early stopping, best iteration is:
[31]	valid_0's multi_error: 0.0775623
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.265928
[100]	valid_0's multi_error: 0.265928
Early stopping, best iteration is:
[20]	valid_0's multi_error: 0.254848
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.263158
[100]	valid_0's multi_error: 0.263158
Early stopping, 

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.177285
[100]	valid_0's multi_error: 0.168975
Early stopping, best iteration is:
[4]	valid_0's multi_error: 0.119114
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.157895
[100]	valid_0's multi_error: 0.160665
Early stopping, best iteration is:
[13]	valid_0's multi_error: 0.138504
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.155125
[100]	valid_0's multi_error: 0.155125
Early stopping, best iteration is:
[8]	valid_0's multi_error: 0.135734
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.252078
[100]	valid_0's multi_error: 0.252078
Early stopping, best iteration is:
[1]	valid_0's multi_error: 0.229917
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.216066
[100]	valid_0's multi_error: 0.216066
Early stopping, best iter

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.263158
[100]	valid_0's multi_error: 0.257618
Early stopping, best iteration is:
[25]	valid_0's multi_error: 0.254848
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.174515
[100]	valid_0's multi_error: 0.171745
Early stopping, best iteration is:
[28]	valid_0's multi_error: 0.152355
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.146814
[100]	valid_0's multi_error: 0.135734
Early stopping, best iteration is:
[30]	valid_0's multi_error: 0.132964
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.124654
[100]	valid_0's multi_error: 0.119114
Early stopping, best iteration is:
[12]	valid_0's multi_error: 0.0914127
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.130194
[100]	valid_0's multi_error: 0.135734
Early stopping, best 

...

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0969529
[100]	valid_0's multi_error: 0.113573
[150]	valid_0's multi_error: 0.113573
Early stopping, best iteration is:
[50]	valid_0's multi_error: 0.0969529
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.102493
[100]	valid_0's multi_error: 0.105263
Early stopping, best iteration is:
[12]	valid_0's multi_error: 0.0886427
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0498615
[100]	valid_0's multi_error: 0.0554017
Early stopping, best iteration is:
[18]	valid_0's multi_error: 0.0360111
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0747922
[100]	valid_0's multi_error: 0.0831025
Early stopping, best iteration is:
[13]	valid_0's multi_error: 0.0581717
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.235457
[100]	valid_0

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.238227
[100]	valid_0's multi_error: 0.243767
Early stopping, best iteration is:
[1]	valid_0's multi_error: 0.210526
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.168975
[100]	valid_0's multi_error: 0.174515
Early stopping, best iteration is:
[23]	valid_0's multi_error: 0.144044
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.160665
[100]	valid_0's multi_error: 0.171745
Early stopping, best iteration is:
[12]	valid_0's multi_error: 0.138504
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.099723
[100]	valid_0's multi_error: 0.105263
Early stopping, best iteration is:
[19]	valid_0's multi_error: 0.0941828
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.157895
[100]	valid_0's multi_error: 0.146814
Early stopping, best i

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.196676
[100]	valid_0's multi_error: 0.199446
Early stopping, best iteration is:
[8]	valid_0's multi_error: 0.174515
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.213296
[100]	valid_0's multi_error: 0.210526
Early stopping, best iteration is:
[11]	valid_0's multi_error: 0.177285
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0858726
[100]	valid_0's multi_error: 0.0914127
Early stopping, best iteration is:
[21]	valid_0's multi_error: 0.0803324
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.113573
[100]	valid_0's multi_error: 0.119114
Early stopping, best iteration is:
[11]	valid_0's multi_error: 0.0941828
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.144044
[100]	valid_0's multi_error: 0.146814
Early stopping, bes

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.132964
[100]	valid_0's multi_error: 0.132964
Early stopping, best iteration is:
[29]	valid_0's multi_error: 0.121884
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0581717
[100]	valid_0's multi_error: 0.0526316
Early stopping, best iteration is:
[11]	valid_0's multi_error: 0.0498615
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0831025
[100]	valid_0's multi_error: 0.0914127
Early stopping, best iteration is:
[41]	valid_0's multi_error: 0.0692521
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0914127
[100]	valid_0's multi_error: 0.0941828
Early stopping, best iteration is:
[9]	valid_0's multi_error: 0.0692521
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0415512
[100]	valid_0's multi_error: 0.0415512
Early stoppi

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.066482
[100]	valid_0's multi_error: 0.0609418
Early stopping, best iteration is:
[5]	valid_0's multi_error: 0.0498615
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0747922
[100]	valid_0's multi_error: 0.0775623
Early stopping, best iteration is:
[40]	valid_0's multi_error: 0.0720222
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0803324
[100]	valid_0's multi_error: 0.0831025
Early stopping, best iteration is:
[21]	valid_0's multi_error: 0.0720222
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.102493
[100]	valid_0's multi_error: 0.108033
Early stopping, best iteration is:
[29]	valid_0's multi_error: 0.0941828
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0831025
[100]	valid_0's multi_error: 0.0803324
[150]	valid_

In [12]:
result_df.to_csv('CSV/song1_result.csv', index=False)